Что делать?

1.Датасет ml-latest

2.Вспомнить подходы, которые мы разбирали

3.Выбрать понравившийся подход к гибридным системам

4.Написать свою


In [108]:
from surprise import Dataset, Reader, KNNBasic, KNNWithMeans
from surprise.model_selection import train_test_split
from surprise.model_selection import KFold
from surprise import accuracy
from tqdm import tqdm_notebook
import pandas as pd
import numpy as np
from scipy.spatial.distance import cityblock, cosine, euclidean, hamming, jaccard, rogerstanimoto, correlation
import scipy.sparse as sparse
from implicit.als import AlternatingLeastSquares

In [114]:
ratings = pd.read_csv('C:/Users/GreenBenito/Desktop/Нетология/Введение и классификация рекомендательных систем/Архив/ratings.csv')

In [115]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


Попробуем каскадную модель:
     - на первом этапе выберем 100 самых похожих юзеров
     - на втором этапе для 100 юзеров сделаем svd

In [121]:
num_items = ratings.movieId.max() # самый большой индекс фильмов

In [120]:
user_vector = {} # векторы пользователей

for user, group in tqdm_notebook(ratings.groupby('userId')): # для каждого пользователя
    user_vector[user] = np.zeros(num_items)                  # создаем вектор нулей
    
    for i in range(len(group.movieId.values)):               # для каждого фильма
        it = group.movieId.values[i]                         # оперделяем индекс
        r = group.rating.values[i]                           # оперделяем рейтинг
        user_vector[user][int(it - 1)] = r                    
        # записываем рейтинг в вектор пользователя на место индекс фильма минус один

C:\Users\GreenBenito\anaconda3\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [158]:
user = 1 # определим самых похожих пользователей для первого юзера

users = []
distances = []

for key in tqdm_notebook(user_vector.keys()): # для каждого юзера
    if key == user:
        continue
    
    users.append(key) # добавляем юзера в список
    distances.append(correlation(user_vector[user], user_vector[key])) 
    # добавляем расстояние между первым и текущим юзером в список

best_indexes = np.argsort(distances)[:100] # выбираем индексы самых похожих
best_users = [users[i] for i in best_indexes]

C:\Users\GreenBenito\anaconda3\lib\site-packages\ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [159]:
ratings['is_best']=ratings['userId'].apply(lambda u: u in best_users or u==user, 1)

In [160]:
new_ratings=ratings.loc[ratings['is_best']==True]

In [161]:
new_ratings.head()

,userId,movieId,rating,timestamp,is_best
0,1,1,4.0,964982703,True
1,1,3,4.0,964981247,True
2,1,6,4.0,964982224,True
3,1,47,5.0,964983815,True
4,1,50,5.0,964982931,True


In [162]:
del new_ratings['timestamp']
del new_ratings['is_best']

In [163]:
users=list(new_ratings.userId.unique())
items=list(new_ratings.movieId.unique())
rat=list(new_ratings.rating)

In [164]:
rows=new_ratings.userId.astype(int)
cols=new_ratings.movieId.astype(int)

In [165]:
df_sparse=sparse.csr_matrix((rat,(cols,rows)))

In [166]:
model=AlternatingLeastSquares(factors=50)

In [167]:
model.fit(df_sparse)

In [168]:
userID=1

user_items=df_sparse.T.tocsr()
recomm=model.recommend(userID, user_items)

In [169]:
recomm

[(858, 0.9796519),
 (1200, 0.9792107),
 (1527, 0.92783684),
 (1221, 0.8921298),
 (3301, 0.87583923),
 (1584, 0.8649537),
 (2167, 0.85213506),
 (4571, 0.84529024),
 (377, 0.836297),
 (3996, 0.83510983)]

Пользователю 1 будем рекомендовать фильмы из recomm